In [ ]:
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from chempy import ReactionSystem
from chempy.units import (
    default_constants,
    default_units as u,
    SI_base_registry as ureg
)
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.rates import SinTemp
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string("""
2 HNO2 -> H2O + NO + NO2; MassAction(EyringHS.fk('dH1', 'dS1'))
2 NO2 -> N2O4; MassAction(EyringHS.fk('dH2', 'dS2'))
""")  # fictitious thermodynamic parameters

In [ ]:
st = SinTemp(unique_keys='Tbase Tamp Tangvel Tphase'.split())

In [ ]:
odesys, extra = get_odesys(rsys, include_params=False, substitutions={'temperature': st},
                           unit_registry=ureg, constants=default_constants)

In [ ]:
init_conc = defaultdict(lambda: 0*u.M, HNO2=1*u.M, H2O=55*u.M)
params = dict(
    Tbase=300*u.K,
    Tamp=10*u.K,
    Tangvel=2*math.pi/(10*u.s),
    Tphase=-math.pi/2,
    dH1=85e3*u.J/u.mol,
    dS1=10*u.J/u.K/u.mol,
    dH2=70e3*u.J/u.mol,
    dS2=20*u.J/u.K/u.mol
)
duration = 60*u.s

In [ ]:
def integrate_and_plot(system):
    result = system.integrate(duration, init_conc, params, integrator='cvode', nsteps=2000)
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    result.plot(names='NO HNO2 N2O4'.split(), ax=axes[0])
    result.plot(names='NO2'.split(), ax=axes[1])
    print({k: v for k, v in sorted(result.info.items()) if not k.startswith('internal')})

In [ ]:
integrate_and_plot(odesys)

In [ ]:
odesys.param_names

In [ ]:
len(odesys.exprs)

In [ ]:
asys = odesys.as_autonomous()
len(asys.exprs)

In [ ]:
[a - o for a, o in zip(asys.exprs[:-1],odesys.exprs)]

In [ ]:
asys.exprs[-1]

In [ ]:
asys.get_jac()[:-1,:-1] - odesys.get_jac()

In [ ]:
import sympy as sym
sym.init_printing()
args = _x, _y, _p = asys.pre_process(*asys.to_arrays(1*u.s, init_conc, params))
args

In [ ]:
asys.f_cb(*args)

In [ ]:
asys.j_cb(*args)

In [ ]:
argsode = odesys.pre_process(*odesys.to_arrays(1*u.s, init_conc, params))
argsode

In [ ]:
argsode[0] - args[0], argsode[1] - args[1][:-1], argsode[2] - args[2]

In [ ]:
odesys.f_cb(*argsode)

In [ ]:
odesys.j_cb(*argsode)

In [ ]:
integrate_and_plot(asys)

In [ ]:
odesys.ny, asys.ny

In [ ]:
asys.pre_process(1, [0,1,2,3,4], [5,6,7,8])